In [1]:
import torch
import random

import torch.nn.functional as F
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
import torch.nn.functional as F
from botorch.acquisition import ExpectedImprovement



m:\workspace\bo\botorch_contrast\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
BATCHSIZE = 20
TARGET = [0.34, 0.2]


#r = torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  
def dif(tensor, target):
  target = torch.tensor([target], dtype=torch.double)
  r = target - tensor 
  r = torch.abs(r)
  r = torch.sum(r)
  # r = torch.clamp(r,0,1)
  return 1-torch.tensor([[r]], dtype=torch.double)
  


def getSample():
  return torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  

X = torch.tensor([], dtype=torch.double)
Y = torch.tensor([], dtype=torch.double)
# Create a Standardize transform using the data

for i in range(BATCHSIZE):
  x = getSample()
  X = torch.cat((X, x), dim=0)
  y = dif(x, TARGET)
  Y = torch.cat((Y, y), dim=0)
  
  

print(Y)


list(zip(X, Y))


tensor([[ 0.6733],
        [ 0.7766],
        [ 0.0821],
        [ 0.9250],
        [ 0.7040],
        [ 0.3356],
        [ 0.6494],
        [ 0.3962],
        [ 0.3809],
        [ 0.3470],
        [ 0.5518],
        [ 0.5548],
        [ 0.1964],
        [ 0.6745],
        [ 0.4687],
        [ 0.5278],
        [ 0.3713],
        [ 0.3435],
        [-0.1649],
        [ 0.7907]], dtype=torch.float64)


[(tensor([0.1446, 0.3313], dtype=torch.float64),
  tensor([0.6733], dtype=torch.float64)),
 (tensor([0.2612, 0.0553], dtype=torch.float64),
  tensor([0.7766], dtype=torch.float64)),
 (tensor([0.5728, 0.8851], dtype=torch.float64),
  tensor([0.0821], dtype=torch.float64)),
 (tensor([0.2805, 0.2155], dtype=torch.float64),
  tensor([0.9250], dtype=torch.float64)),
 (tensor([0.1162, 0.1279], dtype=torch.float64),
  tensor([0.7040], dtype=torch.float64)),
 (tensor([0.9057, 0.2987], dtype=torch.float64),
  tensor([0.3356], dtype=torch.float64)),
 (tensor([0.1795, 0.3902], dtype=torch.float64),
  tensor([0.6494], dtype=torch.float64)),
 (tensor([0.1881, 0.6519], dtype=torch.float64),
  tensor([0.3962], dtype=torch.float64)),
 (tensor([0.1842, 0.6633], dtype=torch.float64),
  tensor([0.3809], dtype=torch.float64)),
 (tensor([0.2172, 0.7303], dtype=torch.float64),
  tensor([0.3470], dtype=torch.float64)),
 (tensor([0.6508, 0.0626], dtype=torch.float64),
  tensor([0.5518], dtype=torch.float64)),

In [23]:
Y =  F.normalize(Y, dim=None, p=2)
train_Y = standardize(Y)
#train_Y = Y
gp = SingleTaskGP(X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

In [24]:

# Define the Expected Improvement (EI) acquisition function
ei = ExpectedImprovement(gp, best_f=torch.max(Y))

# 'best_observed_value' is the best observed value from your optimization process so far
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

# Optimize the EI acquisition function to find the next candidate point
candidate, acq_value = optimize_acqf(
    ei, bounds=bounds, q=1, num_restarts=20, raw_samples=150,
)

print(candidate)

tensor([[0.3215, 0.2052]])
